## Objective 
1. 自動化完全整理好
2. 

## Reference 
```Handmade_from_zero_to_hero.ipynb```

In [1]:
from facebook_business.adobjects.adaccount import AdAccount
from facebook_business.adobjects.adsinsights import AdsInsights
from facebook_business.api import FacebookAdsApi
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import csv
import time
import pandas as pd
import numpy as np
import pyodbc
import logging
import traceback, sys
import smtplib
from email.mime.text import MIMEText
import socket

In [2]:
# 連線SQL
DBdata = {'ip':'192.168.99.142', 'db':'CMAPP', 'user142': 'cmapp', 'pwd142': '0000'}
conn_DBdata = pyodbc.connect('DRIVER={SQL Server};'\
                            'SERVER=' + DBdata['ip'] + \
                            ';DATABASE=' + DBdata['db'] +\
                            ';UID=' + DBdata['user142'] +\
                            ';PWD=' + DBdata['pwd142'] + ';')

DBdata_cr = conn_DBdata.cursor()
TableName = 'FB_AdsData'

## 廣告帳號所有名單

In [15]:
Act_list = {'籌碼K線(原本測試帳號)': '132799196821088',
'StockAiPro APP': '783539402272443',
'StockAiPro APP(iOS)': '1143824496065734',
'icheck': '936847127084019',
'icheck(iOS)': '4196385753706403',
'丹尼爾PC+APP': '452504569395856',
'丹尼爾PC+APP(iOS)': '3871892012868660',
'價值K線APP': '795937830999375',
'價值K線APP(iOS)': '294383082255305',
'優惠王-costco同學會': '712103039460113',
'優惠王-costco同學會(iOS)': '467704917976698',
'大大PM': '509426729992360',
'大股東 APP': '753062955394894',
'大股東 APP(iOS)': '1864362753722473',
'我的房價漲多少': '149018927107258',
'我的房價漲多少(iOS)': '454759162500759',
'房屋價值地圖': '157768706197434',
'房屋價值地圖(iOS)': '785822512339192',
'投資小學堂APP': '219944153210593',
'投資小學堂APP(iOS)': '285019616541316',
'新聞爆料同學會': '129225645812364',
'新聞爆料同學會(iOS)': '462416488363153',
'月老': '505050013831773',
'月老(iOS)': '462146505235011',
'期權先生PC+APP': '197791738405867',
'期權先生PC+APP(iOS)': '449681076323070',
'期貨電子盤 APP': '3873072282807287',
'期貨電子盤 APP(iOS)': '170149104947622',
'林恩如PC+APP': '534983857479217',
'林恩如PC+APP(iOS)': '491297538573013',
'權證小哥PC+APP': '1132572753851821',
'權證小哥PC+APP(iOS)': '436996980921253',
'武財神': '428844138551031',
'武財神(iOS)': '298213318597607',
'每日頭條APP': '222102939378962',
'每日頭條APP(iOS)': '551649459143632',
'無聊詹PC+APP': '440198387247896',
'無聊詹PC+APP(iOS)': '1339779816406536',
'租屋雷達': '1086617258501383',
'租屋雷達(iOS)': '1061710770977404',
'算利教官 APP': '480153816358865',
'算利教官 APP(iOS)': '4284601198239102',
'籌碼K線 APP': '372330733913901',
'籌碼K線 APP (iOS)': '136443351648696',
'籌碼K線 PC': '3648034708655701',
'美股夢想家 PC+APP': '279643700237508',
'美股夢想家 PC+APP(iOS)': '478730439945199',
'股人阿勳 APP': '1614448442086305',
'股人阿勳 APP(iOS)': '830300951168633',
'股市Podcast': '448233149813034',
'股市Podcast(iOS)': '1329909827384466',
'股市爆料同學會APP': '238268321010982',
'股市爆料同學會APP(iOS)': '1757870844381494',
'自由人PC+APP': '137512381633886',
'自由人PC+APP(iOS)': '166308061984091',
'艾蜜莉 APP': '319046682694239',
'艾蜜莉 APP(iOS)': '2813564628898257',
'華倫 APP': '476405876831222',
'華倫 APP(iOS)': '426164802018260',
'阿格力 APP': '2767955253458688',
'阿格力 APP(iOS)': '293950655476450',
'阿水PC+APP': '787416052182106',
'阿水PC+APP(iOS)': '278212370485576',
'陳重銘APP': '268527648070343',
'陳重銘APP(iOS)': '458094148869336',
}
# print(Act_list.values())

for key, value in Act_list.items():
    print(value)

132799196821088
783539402272443
1143824496065734
936847127084019
4196385753706403
452504569395856
3871892012868660
795937830999375
294383082255305
712103039460113
467704917976698
509426729992360
753062955394894
1864362753722473
149018927107258
454759162500759
157768706197434
785822512339192
219944153210593
285019616541316
129225645812364
462416488363153
505050013831773
462146505235011
197791738405867
449681076323070
3873072282807287
170149104947622
534983857479217
491297538573013
1132572753851821
436996980921253
428844138551031
298213318597607
222102939378962
551649459143632
440198387247896
1339779816406536
1086617258501383
1061710770977404
480153816358865
4284601198239102
372330733913901
136443351648696
3648034708655701
279643700237508
478730439945199
1614448442086305
830300951168633
448233149813034
1329909827384466
238268321010982
1757870844381494
137512381633886
166308061984091
319046682694239
2813564628898257
476405876831222
426164802018260
2767955253458688
293950655476450
78741605

## 從 SQL 轉換成 pd.DF

In [4]:
def Get_SN_RD(TableName):
    get_both = """
        SELECT TOP(10)*
        FROM [CMAPP].[dbo].[{}]
        """
    print(get_both.format(TableName))
    DBdata_cr.execute(get_both.format(TableName))
    print([x[0] for x in DBdata_cr.description])
    print([row for row in DBdata_cr.fetchall()])
    
    df = pd.DataFrame(DBdata_cr.fetchall())
#     df.columns = [x[0] for x in DBdata_cr.description]
    conn_DBdata.commit()
    return df

df = Get_SN_RD(TableName)
df


        SELECT TOP(10)*
        FROM [CMAPP].[dbo].[FB_AdsData]
        
['Serial_Number', 'Campaign_Name', 'Adset_Name', 'Ad_Name', 'Objective', 'Amount_Spent', 'Amount_Install', 'Amount_Purchase', 'Purchase_Conversion_Value', 'Amount_Subscribe', 'Subscribe_Conversion_Value', 'CPM', 'CTR', 'Impressions', 'Reporting_Date', 'Category']
[(Decimal('202001010000'), '期貨電子盤_下載_IOS(舊)', '興趣期貨受眾_IOS', '創意廣告', 'APP_INSTALLS', Decimal('139'), Decimal('7'), Decimal('1'), Decimal('8888'), None, None, Decimal('57.17812'), Decimal('1.19293'), Decimal('2431'), '20200101', 'FB'), (Decimal('202001010001'), '期貨電子盤_下載_IOS(舊)', '期貨電子盤類似受眾_IOS', '創意廣告', 'APP_INSTALLS', Decimal('124'), Decimal('6'), None, None, None, None, Decimal('57.94393'), Decimal('0.79439'), Decimal('2140'), '20200101', 'FB'), (Decimal('202001010002'), '籌碼K線_ 競價下載_行銷活動花費無限制', '理財寶相關受眾_出價上限35元_圖片_KUSO畫面_android', '圖片', 'APP_INSTALLS', Decimal('99'), Decimal('9'), None, None, None, None, Decimal('85.05155'), Decimal('1.89003'), Decimal(

""


In [5]:
def Get_SN_RD(TableName):
    get_both = """
        SELECT [Serial_Number], [Reporting_Date]
        FROM [CMAPP].[dbo].[{}]
        WHERE  [Serial_Number] = (SELECT MAX(Serial_Number) 
        FROM [CMAPP].[dbo].[{}])
        """
    print(get_both.format(TableName, TableName))
    DBdata_cr.execute(get_both.format(TableName, TableName))
    # 獲取最近的日期(p)和流水號(Both_SN_RD)
    Last_SN, Last_RD = DBdata_cr.fetchall()[0]
    print(Last_SN, Last_RD, end = '\n\n')
    
    df = pd.DataFrame(DBdata_cr.fetchall())
    print(DBdata_cr.description, end = '\n\n')
#     df.columns = [ x for x in DBdata_cr.description ]
    conn_DBdata.commit()
    return Last_SN, Last_RD, df

Last_SN, Last_RD, df = Get_SN_RD(TableName)
# print(a[0][1])
# print(a[0][0])
df


        SELECT [Serial_Number], [Reporting_Date]
        FROM [CMAPP].[dbo].[FB_AdsData]
        WHERE  [Serial_Number] = (SELECT MAX(Serial_Number) 
        FROM [CMAPP].[dbo].[FB_AdsData])
        
2021032800053028 20210328

(('Serial_Number', <class 'decimal.Decimal'>, None, 18, 18, 0, False), ('Reporting_Date', <class 'str'>, None, 20, 20, 0, False))



""


In [6]:
# 提取流水號、報告日期
def Get_SN_RD(TableName):
    get_both = """
        SELECT [Serial_Number], [Reporting_Date]
        FROM [CMAPP].[dbo].[{}]
        WHERE  [Serial_Number] = (SELECT MAX(Serial_Number) 
        FROM [CMAPP].[dbo].[{}])
        """
    print(get_both.format(TableName, TableName))
    DBdata_cr.execute(get_both.format(TableName, TableName))
    Both_SN_RD = DBdata_cr.fetchall()
    conn_DBdata.commit()
    return Both_SN_RD


# 將Reporting_Date回溯1個月前
def RD_date_back(cleaned_RD):
    # 將str變成時間格式(datetime)，才能做計算
    reporting_date = datetime.strptime(str(cleaned_RD), '%Y%m%d')
    
    # 當現在的日期已經over
    if reporting_date.date() < (datetime.now().date() - relativedelta(months = 1)):
        reporting_date = reporting_date.date()
    else:
        # 將日期回溯1個月
        reporting_date = reporting_date.date() - relativedelta(months = 1)
    return reporting_date


# 增加流水號 & 廣告類別
def Expand_df(df, cleaned_SN): 
    # 第幾排插入流水號 
    idx_SN = 0
    idx_CA = 11
    Serial_Number = []
    # 廣告類別
    Category = 'FB'

    # 增加Serial_Number & Catergory(FB, GOOGL, AAPL)
    for order in range(len(df)):
        s_num = str(df.Reporting_Date[order]) + '000' + str(int(str(cleaned_SN)[8:]) + order + 1)
        Serial_Number.append(int(s_num))
    
    # 將資料插入
    df.insert(loc = idx_SN, column = 'Serial_Number', value = Serial_Number)
    df.insert(loc = idx_CA, column = 'Category', value = Category)
    return df


# 將CSV內檔案的
def None_to_Null(df):
    for col in df.columns:
        df[col] = df[col].apply(lambda x: None if x == '' else x)
    return df 


# 將df的資料備份下來
def Df_to_Csv(df):
    # 將df內的Data儲存成CSV file
    df.to_csv('{}_FBAdsTest.csv'.format(datetime.datetime.now().strftime('%Y%m%d')), index = False, encoding='utf_8_sig')
    return None


# 刪除資料
def DelSQL(TableName, start_time, now_time):
    str_query = """
        DELETE
        FROM [CMAPP].[dbo].[{}]
        WHERE Reporting_Date BETWEEN {} AND {}
        """
    print(str_query.format(TableName, start_time, now_time))
    DBdata_cr.execute(str_query.format(TableName, start_time, now_time))
    conn_DBdata.commit()   


# 寫入資料
def InsertSQL(df, TableName):
    str_query = """
        INSERT INTO [CMAPP].[dbo].[{}]([Serial_Number],[Campaign_Name],[Adset_Name],[Ad_Name],[Objective],[Amount_Spent],[Amount_Install],\
        [Amount_Purchase],[Purchase_Conversion_Value],[Amount_Subscribe],[Subscribe_Conversion_Value],[CPM],[CTR],[Impressions],[Reporting_Date]\
        ,[Category])VALUES({})
        """
    # 將csv DataFrame每筆資料變成list形式
    insert_data = df.values.tolist()
    for i in range(len(insert_data)):
#         print(str_query.format(TableName, str(insert_data[i]).strip('[]').replace('None', 'null')))      
        DBdata_cr.execute(str_query.format(TableName, str(insert_data[i]).strip('[]').replace('None', 'null')))
        conn_DBdata.commit() 